In [4]:
import numpy as np

class GeneticAlgorithmConstrained:
    """
    Genetic Algorithm (GA) for constrained continuous / mixed-integer optimization (minimization).

    - Feasible-only: only individuals satisfying constraint_func(x) are allowed.
    - Mixed variables: bounds = [(min, max, "any" or "int"), ...]
    """

    def __init__(
        self,
        objective_func,
        bounds,
        constraint_func=None,
        pop_size=30,
        pc=0.9,              # crossover probability
        pm=0.1,              # mutation probability (per variable)
        max_gen=1000,
        tourn_size=2,
        max_resample=10000,
        seed=5
    ):
        """
        Parameters
        ----------
        objective_func : callable
            Function f(x) to minimize. x is 1D numpy array.
        bounds : list of tuple
            [(x1_min, x1_max, "any" or "int"), ...]
            If 3rd element omitted, "any" is assumed.
        constraint_func : callable or None
            g(x) -> bool, True if feasible.
        pop_size : int
            Population size.
        pc : float
            Crossover probability.
        pm : float
            Mutation probability (per variable).
        max_gen : int
            Number of generations.
        tourn_size : int
            Tournament size for selection.
        max_resample : int
            Max attempts to resample a feasible individual.
        seed : int or None
            Random seed.
        """
        self.objective_func = objective_func
        self.constraint_func = constraint_func

        self.dim = len(bounds)
        self.bounds = np.array([(b[0], b[1]) for b in bounds], dtype=float)
        self.types = [(b[2].lower() if len(b) > 2 else "any") for b in bounds]

        self.pop_size = pop_size
        self.pc = pc
        self.pm = pm
        self.max_gen = max_gen
        self.tourn_size = tourn_size
        self.max_resample = max_resample

        self.rng = np.random.default_rng(seed)

        # Variable-wise span (for mutation step size)
        self.span = self.bounds[:, 1] - self.bounds[:, 0]

        # Population & fitness
        self.P = None              # shape: (pop_size, dim)
        self.f = None              # shape: (pop_size,)
        self.best_x = None
        self.best_f = None
        self.history_best_f = []

    # ------------------- Helpers -------------------
    def _is_feasible(self, x):
        if self.constraint_func is None:
            return True
        return bool(self.constraint_func(x))

    def _sample_random_individual(self):
        """Sample a single feasible individual."""
        for _ in range(self.max_resample):
            x = self.rng.uniform(self.bounds[:, 0], self.bounds[:, 1])
            # integer rounding
            for j in range(self.dim):
                if self.types[j] == "int":
                    x[j] = round(x[j])
            x = np.clip(x, self.bounds[:, 0], self.bounds[:, 1])
            if self._is_feasible(x):
                return x
        raise RuntimeError("GA: Failed to sample a feasible individual.")

    def _initialize_population(self):
        self.P = np.empty((self.pop_size, self.dim))
        for i in range(self.pop_size):
            self.P[i] = self._sample_random_individual()
        self._evaluate_population()

    def _evaluate_population(self):
        self.f = np.apply_along_axis(self.objective_func, 1, self.P)
        best_idx = np.argmin(self.f)
        self.best_x = self.P[best_idx].copy()
        self.best_f = self.f[best_idx]

    # ------------------- Selection -------------------
    def _tournament_select(self):
        """Return index of selected individual using tournament selection."""
        idxs = self.rng.integers(0, self.pop_size, size=self.tourn_size)
        best_idx = idxs[np.argmin(self.f[idxs])]
        return best_idx

    # ------------------- Crossover -------------------
    def _crossover_blx(self, p1, p2, alpha=0.5):
        """
        BLX-alpha crossover for real-coded GA.
        Returns two children.
        """
        c1 = p1.copy()
        c2 = p2.copy()
        for j in range(self.dim):
            if self.rng.random() < self.pc:
                x_min = min(p1[j], p2[j])
                x_max = max(p1[j], p2[j])
                d = x_max - x_min
                lower = x_min - alpha * d
                upper = x_max + alpha * d
                # sample within expanded range, then clip to global bounds
                val1 = self.rng.uniform(lower, upper)
                val2 = self.rng.uniform(lower, upper)

                # type handling
                if self.types[j] == "int":
                    val1 = round(val1)
                    val2 = round(val2)

                c1[j] = np.clip(val1, self.bounds[j, 0], self.bounds[j, 1])
                c2[j] = np.clip(val2, self.bounds[j, 0], self.bounds[j, 1])
        return c1, c2

    # ------------------- Mutation -------------------
    def _mutate(self, x):
        """
        Uniform mutation within a small step proportional to variable span.
        """
        y = x.copy()
        for j in range(self.dim):
            if self.rng.random() < self.pm:
                step = 0.1 * self.span[j]  # 10% of range
                val = y[j] + self.rng.uniform(-step, step)
                if self.types[j] == "int":
                    val = round(val)
                y[j] = np.clip(val, self.bounds[j, 0], self.bounds[j, 1])
        return y

    # ------------------- Feasibility repair -------------------
    def _repair_feasible(self, x):
        """
        If x is infeasible, try to mutate/resample until feasible.
        """
        if self._is_feasible(x):
            return x
        # try some mutations
        y = x.copy()
        for _ in range(self.max_resample // 2):
            y = self._mutate(y)
            if self._is_feasible(y):
                return y
        # fallback: resample a random feasible individual
        return self._sample_random_individual()

    # ------------------- Main GA loop -------------------
    def run(self, verbose=False):
        """
        Run GA optimization.

        Returns
        -------
        best_x : numpy.ndarray
            Best solution found.
        best_f : float
            Best objective value.
        history_best_f : list of float
            Best f over generations.
        """
        self._initialize_population()
        self.history_best_f = [self.best_f]

        for gen in range(1, self.max_gen + 1):
            new_P = []

            # Elitism: keep current best
            elite_idx = np.argmin(self.f)
            elite = self.P[elite_idx].copy()
            new_P.append(elite)

            # Generate rest of population via selection + crossover + mutation
            while len(new_P) < self.pop_size:
                i1 = self._tournament_select()
                i2 = self._tournament_select()
                p1, p2 = self.P[i1], self.P[i2]

                c1, c2 = self._crossover_blx(p1, p2)

                c1 = self._mutate(c1)
                c2 = self._mutate(c2)

                c1 = self._repair_feasible(c1)
                c2 = self._repair_feasible(c2)

                new_P.append(c1)
                if len(new_P) < self.pop_size:
                    new_P.append(c2)

            self.P = np.array(new_P)
            self._evaluate_population()
            self.history_best_f.append(self.best_f)

            if verbose and (gen % max(1, self.max_gen // 10) == 0):
                print(f"Gen {gen:6d} | Best f = {self.best_f:.6e}")

        return self.best_x, self.best_f, self.history_best_f

In [5]:
# ==========================================================
# Example 1: Tension/compression spring design problem
# ==========================================================

def spring_obj(x):
    """
    Tension/compression spring design objective function.
    x: [x1, x2, x3]
    f(x) = (x3 + 2) * x2 * x1^2
    """
    x = np.asarray(x, dtype=float)
    x1, x2, x3 = x
    return (x3 + 2.0) * x2 * x1**2


def spring_cons(x):
    """
    Spring design constraints g1..g4(x) <= 0.
    Returns True if all constraints are satisfied.
    """
    x = np.asarray(x, dtype=float)
    x1, x2, x3 = x

    # g1(x) = 1 - (x2^3 * x3) / (71,785 x1^4) <= 0
    g1 = 1.0 - (x2**3 * x3) / (71_785.0 * x1**4)

    # g2(x) = (4x2^2 - x1 x2) / (12,566 (x2 x1^3 - x1^4)) + 1/(5,108 x1^2) - 1 <= 0
    denom = 12_566.0 * (x2 * x1**3 - x1**4)
    if abs(denom) < 1e-12:
        return False
    g2 = (4.0 * x2**2 - x1 * x2) / denom + 1.0 / (5_108.0 * x1**2) - 1.0

    # g3(x) = 1 - 140.45 x1 / (x2^2 x3) <= 0
    g3 = 1.0 - (140.45 * x1) / (x2**2 * x3)

    # g4(x) = (x1 + x2) / 1.5 - 1 <= 0
    g4 = (x1 + x2) / 1.5 - 1.0

    return (g1 <= 0.0) and (g2 <= 0.0) and (g3 <= 0.0) and (g4 <= 0.0)


bounds_spring = [
    (0.05, 2.0, "any"),   # x1
    (0.25, 1.3, "any"),   # x2
    (2.0, 15.0, "any")    # x3
]


# ==========================================================
# Example 2: Pressure vessel design problem
# ==========================================================

def pressure_vessel_obj(x):
    """
    Pressure vessel design objective function.
    x: [x1, x2, x3, x4]
    f(x) = 0.6224 x1 x3 x4 + 1.7781 x2 x3^2 + 3.1661 x1^2 x4 + 19.84 x1^2 x3
    """
    x = np.asarray(x, dtype=float)
    x1, x2, x3, x4 = x

    return (
        0.6224 * (0.0625*x1) * x3 * x4 + 1.7781 * (0.0625*x2) * x3**2 + 3.1661 * (0.0625*x1)**2 * x4 + 19.84  * (0.0625*x1)**2 * x3
    )


def pressure_vessel_cons(x):
    """
    Pressure vessel design constraints g1..g4(x) <= 0.
    Returns True if all constraints are satisfied.
    """
    x = np.asarray(x, dtype=float)
    x1, x2, x3, x4 = x

    # g1(x) = -x1 + 0.0193 x3 <= 0
    g1 = -0.0625*x1 + 0.0193 * x3

    # g2(x) = -x2 + 0.00954 x3 <= 0
    g2 = -0.0625*x2 + 0.00954 * x3

    # g3(x) = -pi x3^2 x4 - (4/3) pi x3^3 + 1,296,000 <= 0
    g3 = -np.pi * x3**2 * x4 - (4.0 / 3.0) * np.pi * x3**3 + 1_296_000.0

    # g4(x) = x4 - 240 <= 0
    g4 = x4 - 240.0

    return (g1 <= 0.0) and (g2 <= 0.0) and (g3 <= 0.0) and (g4 <= 0.0)


bounds_pressure_vessel = [
    (1.0, 99.0, "int"),  # x1 (thickness, integer multiple of 0.0625 if scaled)
    (1.0, 99.0, "int"),  # x2
    (10.0,   200.0,        "any"),   # x3
    (10.0,   200.0,        "any"),   # x4
]

# ==========================================================
# Example 3: 3bar truss design
# ==========================================================
def truss_obj(x):
    """
    Objective function:
        f(x) = (2 * sqrt(2) * x1 + x2) * L
    x : [x1, x2]
    """
    x = np.asarray(x, dtype=float)
    x1, x2 = x

    return (2.0 * np.sqrt(2.0) * x1 + x2) * 100

def truss_cons(x):
    """
    Constraint set:
        g1(x) = ((sqrt(2)*x1 + x2) / sqrt(2*x1^2 + 2*x1*x2)) * P - SIGMA <= 0
        g2(x) = (x2 / sqrt(2*x1^2 + 2*x1*x2)) * P - SIGMA <= 0
        g3(x) = (1 / sqrt(2*x2 + x1)) * P - SIGMA <= 0

    Returns True if all constraints are satisfied.
    """
    x = np.asarray(x, dtype=float)
    x1, x2 = x

    # Common denominator term for g1, g2
    denom = np.sqrt(2.0) * x1**2 + 2.0 * x1 * x2
    if denom < 1e-12:
        # Avoid division by zero -> treat as infeasible
        return False

    g1 = ((np.sqrt(2.0) * x1 + x2) / denom) * 2 - 2
    g2 = (x2 / denom) * 2 - 2

    # Denominator for g3
    denom3 = np.sqrt(2.0) * x2 + x1
    if denom3 < 1e-12:
        return False

    g3 = (1.0 / denom3) * 2 - 2

    return (g1 <= 0.0) and (g2 <= 0.0) and (g3 <= 0.0)

bounds_truss = [
    (0.0, 1.0, "any"),  # x1
    (0.0, 1.0, "any")   # x2
]

# ==========================================================
# Example 4: speed reducer optimization
# ==========================================================

def speed_obj(x):
    x = np.asarray(x, dtype=float)
    x1, x2, x3, x4, x5, x6, x7 = x

    return (
        0.7854 * x1 * x2**2 * (3.3333 * x3**2 + 14.9334 * x3 - 43.0934)
        - 1.508 * x1 * (x6**2 + x7**2)
        + 7.4777 * (x6**3 + x7**3)
        + 0.7854 * (x4 * x6**2 + x5 * x7**2)
    )

def speed_cons(x):
    """
    Return True if all constraints g1..g11(x) <= 0.
    """
    x = np.asarray(x, dtype=float)
    x1, x2, x3, x4, x5, x6, x7 = x

    # g1 ~ g4
    g1 = 27.0 / (x1 * x2**2 * x3) - 1.0
    g2 = 397.5 / (x1 * x2**2 * x3**2) - 1.0
    g3 = (1.93 * x4**3) / (x2 * x3 * x6**4) - 1.0
    g4 = (1.93 * x5**3) / (x2 * x3 * x7**4) - 1.0

    # g5, g6
    g5 = (1.0 / (110.0 * x6**3)) * np.sqrt(((745.0 * x4) / (x2 * x3))**2 + 16.9e6) - 1.0
    g6 = (1.0 / (85.0 * x7**3)) * np.sqrt(((745.0 * x5) / (x2 * x3))**2 + 157.5e6) - 1.0

    # g7 ~ g11
    g7  = (x2 * x3) / 40.0 - 1.0
    g8  = (5.0 * x2) / x1 - 1.0
    g9  = (x1 / (12.0 * x2)) - 1.0
    g10 = (1.5 * x6 + 1.9) / x4 - 1.0
    g11 = (1.1 * x7 + 1.9) / x5 - 1.0

    return (
        g1 <= 0 and g2 <= 0 and g3 <= 0 and g4 <= 0 and
        g5 <= 0 and g6 <= 0 and g7 <= 0 and g8 <= 0 and g9 <= 0 and g10 <= 0 and g11 <= 0
    )

bounds_speed = [
    (2.6, 3.6, "any"),   # x1
    (0.7, 0.8, "any"),   # x2
    (17, 28, "int"), # x3
    (7.3, 8.3, "any"),   # x4
    (7.8, 8.3, "any"),   # x5
    (2.9, 3.9, "any"),   # x6
    (5.0, 5.5, "any"),   # x7
]

In [10]:
ga = GeneticAlgorithmConstrained(
    objective_func=speed_obj,
    bounds=bounds_speed,
    constraint_func=speed_cons
)
ga_best_x, ga_best_f, ga_hist = ga.run(verbose=True)

print("Best feasible solution:", ga_best_x)
print("Best objective value:", ga_best_f)

Gen    100 | Best f = 3.004825e+03
Gen    200 | Best f = 3.001386e+03
Gen    300 | Best f = 2.998975e+03
Gen    400 | Best f = 2.998704e+03
Gen    500 | Best f = 2.998238e+03
Gen    600 | Best f = 2.997637e+03
Gen    700 | Best f = 2.996884e+03
Gen    800 | Best f = 2.996884e+03
Gen    900 | Best f = 2.996877e+03
Gen   1000 | Best f = 2.996877e+03
Best feasible solution: [ 3.50012617  0.7        17.          7.3         7.8         3.35117918
  5.2870509 ]
Best objective value: 2996.877420126266
